In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ipywidgets import HBox, VBox

from google.colab import output
output.enable_custom_widget_manager()

df_force = pd.read_csv('ForceData_mat_2.csv', sep=';')
df_pressure = pd.read_csv('PressureData_mat_2.csv', sep=';')
df_strain = pd.read_csv('StrainData_mat_2.csv', sep=';')

marker_shape_map_printing_process = {
    "FDM-a": "square-open",  # Changed to open
    "FDM-b": "square",      # Changed to closed
    "DIW-a": "triangle-up-open",  # Changed to open
    "DIW-b": "triangle-up",      # Changed to closed
    "FDM/DIW-a": "circle-open",  # Changed to open
    "FDM/DIW-b": "circle",      # Changed to closed
    "AJP-a": "triangle-down-open",  # Changed to open
    "AJP-b": "triangle-down",      # Changed to closed
    "SLA-a": "diamond-open",  # Changed to open
    "SLA-b": "diamond",      # Changed to closed
    "DLP-a": "pentagon-open",  # Changed to open
    "DLP-b": "pentagon",      # Changed to closed
    "SLS-a": "hexagon-open",  # Changed to open
    "SLS-b": "hexagon",      # Changed to closed
    # Add more if needed
}

marker_shape_map_sensing_basis = {
    "Material-a": "square",
    "Material-b": "square-open",
    "Auxetic-a": "triangle-up",
    "Auxetic-b": "triangle-up-open",
    "Surface-a": "circle",
    "Surface-b": "circle-open",
    "Porous-a": "triangle-down",
    "Porous-b": "triangle-down-open",
    "Microcracks-a": "diamond",
    "Microcracks-b": "diamond-open",
    "Displacement-a": "pentagon",
    "Displacement-b": "pentagon-open",
    "Alignment-a": "hexagon",
    "Alignment-b": "hexagon-open",
    "Other-a": "cross",  # Changed to 'cross'
    "Other-b": "cross-open",  # Changed to 'cross-open
    # Add more if needed
}

color_map = {
    1: "#648fff",  # Carbon
    2: "#785ef0",  # Metal
    3: "#dc267f",  # Liquid
    4: "#fe6100",  # Wire
    5: "#ffb000",  # Other
}

sensor_map_names = {
        1: "Piezoresistive",
        2: "Capacitive",
        3: "Inductive",
        4: "Frequency",
        5: "Transistor"
    }

material_map_names = {
        1: "Carbon",
        2: "Metal",
        3: "Liquid",
        4: "Wire",
        5: "Other"
    }

def get_color_from_map(value, color_dict):
    return color_dict.get(value, "#cccccc")

def get_plotting_data(df):
    x_data = df.iloc[:, 8] - df.iloc[:, 7]
    y_data = df.iloc[:, 10]
    ele_data = df.iloc[:, 2] #Numeric electrical principle
    mat_data = df.iloc[:, 13] #Numeric mat principle
    print_data = df.iloc[:, 15] #Concat printing string
    sens_data = df.iloc[:, 16] #Concat

    return [x_data, y_data, ele_data, mat_data, print_data, sens_data]

def get_fig(data_list,xlabel,ylabel):
    printing_processes = data_list[4].unique()
    electrical_principles = data_list[2].unique()

    sensing_basis = data_list[5].unique()
    materials = data_list[3].unique()

    fig_left = go.Figure()
    fig_right = go.Figure()

    # Create separate traces for each printing process and electrical principle
    traces_left = []
    for process in printing_processes:
        for principle in electrical_principles:
            filtered_data = data_list[0][(data_list[4] == process) & (data_list[2] == principle)], data_list[1][(data_list[4] == process) & (data_list[2] == principle)]
            if len(filtered_data[0]) > 0:  # Check if there's data for this combination
                if process[-1] == 'a':
                  submark_string = ' (post processing)'
                else:
                  submark_string = ''
                trace = go.Scatter(
                    x=filtered_data[0],
                    y=filtered_data[1],
                    mode='markers',
                    marker=dict(
                        size=10,
                        symbol=marker_shape_map_printing_process.get(process, 'circle'),
                        color=get_color_from_map(principle, color_map),
                        line=dict(width=2, color='black')
                    ),
                    name=f'{process[0:-2]}',  # Only show process in legend (remove -a/b)
                    legendgroup=f'{sensor_map_names[principle]}',  # Group by principle
                    legendgrouptitle=dict(text=f'{sensor_map_names[principle]}'),
                    visible=True,
                    hovertemplate = "DOI: " + "<br>Printing process: " + str(process[0:-2]) + "</br>Electrical principle: " + str(sensor_map_names[principle]) + submark_string + "<extra></extra>"
                )
                traces_left.append(trace)
                fig_left.add_trace(trace)

    traces_right = []
    for basis in sensing_basis:
        for mat in materials:
            filtered_data = data_list[0][(data_list[5] == basis) & (data_list[3] == mat)], data_list[1][(data_list[5] == basis) & (data_list[3] == mat)]
            if basis[-1]=='a':
              submark_string = ''
            else:
              submark_string = ' (Custom)'
            if len(filtered_data[0]) > 0:  # Check if there's data for this combination
                trace = go.Scatter(
                    x=filtered_data[0],
                    y=filtered_data[1],
                    mode='markers',
                    marker=dict(
                        size=10,
                        symbol=marker_shape_map_sensing_basis.get(basis, 'circle'),
                        color=get_color_from_map(mat, color_map),
                        line=dict(width=2, color='black')
                    ),
                    name=f'{basis[0:-2]}',  # Only show basis in legend (remove -a/b)
                    legendgroup=f'{material_map_names[mat]}',  # Group by material
                    legendgrouptitle=dict(text=f'{material_map_names[mat]}'),
                    visible=True,
                    hovertemplate = "DOI: " + "<br>Sensing basis: " + str(basis[0:-2]) + "</br>Material: " + str(material_map_names[mat]) + submark_string + "<extra></extra>"
                )
                traces_right.append(trace)
                fig_right.add_trace(trace)

    # Update layout for legend columns
    fig_left.update_layout(
        legend=dict(
            orientation="h",
            tracegroupgap=150,  # Adjust spacing as needed
            entrywidth=0.23,       # Adjust width as needed
            entrywidthmode="fraction" ,
            y=-0.12, yanchor="top",
        ),
        legend_groupclick="toggleitem",
    )

    fig_right.update_layout(
        legend=dict(
            orientation="h",
            tracegroupgap=150,  # Adjust spacing as needed
            entrywidth=0.23,       # Adjust width as needed
            entrywidthmode="fraction" ,
            y=-0.12, yanchor="top",
        ),
        legend_groupclick="toggleitem",
    )


    # Log scale axes
    fig_left.update_xaxes(title_text=xlabel,type='log')
    fig_left.update_yaxes(title_text=ylabel,type='log')
    fig_right.update_xaxes(title_text=xlabel,type='log')
    fig_right.update_yaxes(title_text=ylabel,type='log')

    fig_left.update_layout(
        width = 700,
        height = 700,
        title = "Printing process, Electrical principle and Direct printing",
    )
    fig_right.update_layout(
        width = 700,
        height = 700,
        title = "Conductive Material, Sensing Basis and Custom material",
    )
    fig_left.update_layout(margin_autoexpand=False,margin_b=250)
    fig_right.update_layout(margin_autoexpand=False,margin_b=250)

    f_left = go.FigureWidget(fig_left)
    f_right = go.FigureWidget(fig_right)

    fig_output=  HBox([f_left, f_right])
    return fig_output

In [ ]:
force = get_plotting_data(df_force)
fig_force = get_fig(force,"Force [N]","Sensitivity [N<sup>-1</sup>]")
fig_force

    'data': [{'hovertemplate': ('DOI: <br>Printing process: FDM' ... ' Piezoresi…

In [ ]:
pressure = get_plotting_data(df_pressure)
fig_pressure = get_fig(pressure,"Pressure [kPa]","Sensitivity [kPa<sup>-1</sup>]")
fig_pressure

    'data': [{'hovertemplate': ('DOI: <br>Printing process: FDM' ... ' Piezoresi…

In [ ]:
strain = get_plotting_data(df_strain)
fig_strain = get_fig(strain,"Strain","Gauge factor")
fig_strain

    'data': [{'hovertemplate': ('DOI: <br>Printing process: FDM' ... 'ost proces…

In [ ]:
# OLD CODE

import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ipywidgets import HBox, VBox

from google.colab import output
output.enable_custom_widget_manager()

# 2) Read the CSV files into pandas DataFrames.
#    Make sure these paths match where your files are.
df_force = pd.read_csv('ForceData_mat_2.csv', sep=';')
df_pressure = pd.read_csv('PressureData_mat_2.csv', sep=';')
df_strain = pd.read_csv('StrainData_mat_2.csv', sep=';')

#------------------------------------------------------------------------------
# PART A: DEFINE MAPPINGS FOR MARKER SHAPES AND COLOR INDICES
#------------------------------------------------------------------------------

# Example marker-shape map for the *left column* subplots (a, b, c).
# Suppose your CSV has a column that stores values like "FDM-a", "DIW-a", etc.
marker_shape_map_printing_process = {
    "FDM-a": "square-open",  # Changed to open
    "FDM-b": "square",      # Changed to closed
    "DIW-a": "triangle-up-open",  # Changed to open
    "DIW-b": "triangle-up",      # Changed to closed
    "FDM/DIW-a": "circle-open",  # Changed to open
    "FDM/DIW-b": "circle",      # Changed to closed
    "AJP-a": "triangle-down-open",  # Changed to open
    "AJP-b": "triangle-down",      # Changed to closed
    "SLA-a": "diamond-open",  # Changed to open
    "SLA-b": "diamond",      # Changed to closed
    "DLP-a": "pentagon-open",  # Changed to open
    "DLP-b": "pentagon",      # Changed to closed
    "SLS-a": "hexagon-open",  # Changed to open
    "SLS-b": "hexagon",      # Changed to closed
    # Add more if needed
}

# Example marker-shape map for the *right column* subplots (d, e, f).
# Suppose your CSV has values like "Material-a", "Auxetic-b", etc.
marker_shape_map_sensing_basis = {
    "Material-a": "square",
    "Material-b": "square-open",
    "Auxetic-a": "triangle-up",
    "Auxetic-b": "triangle-up-open",
    "Surface-a": "circle",
    "Surface-b": "circle-open",
    "Porous-a": "triangle-down",
    "Porous-b": "triangle-down-open",
    "Microcracks-a": "diamond",
    "Microcracks-b": "diamond-open",
    "Displacement-a": "pentagon",
    "Displacement-b": "pentagon-open",
    "Alignment-a": "hexagon",
    "Alignment-b": "hexagon-open",
    "Other-a": "cross",  # Changed to 'cross'
    "Other-b": "cross-open",  # Changed to 'cross-open
    # Add more if needed
}

#------------------------------------------------------------------------------
# PART B: DEFINE MAPPINGS FOR COLOR (SENSOR PRINCIPLE / MATERIAL)
#------------------------------------------------------------------------------

color_map = {
    1: "#648fff",  # Carbon
    2: "#785ef0",  # Metal
    3: "#dc267f",  # Liquid
    4: "#fe6100",  # Wire
    5: "#ffb000",  # Other
}

sensor_map_names = {
        1: "Piezoresistive",
        2: "Capacitive",
        3: "Inductive",
        4: "Frequency",
        5: "Transistor"
    }

material_map_names = {
        1: "Carbon",
        2: "Metal",
        3: "Liquid",
        4: "Wire",
        5: "Other"
    }

# We’ll create a helper function to get color from a dictionary:
def get_color_from_map(value, color_dict):
    return color_dict.get(value, "#cccccc")  # fallback color if not in dict

#------------------------------------------------------------------------------
# PART C: BUILD THE FIGURES
#------------------------------------------------------------------------------

fig_left = go.Figure()
fig_right = go.Figure()

# Modified add_scatter_trace function to associate traces with printing process
def get_plotting_data(df):
    x_data = df.iloc[:, 8] - df.iloc[:, 7]
    y_data = df.iloc[:, 10]
    ele_data = df.iloc[:, 2] #Numeric electrical principle
    mat_data = df.iloc[:, 13] #Numeric mat principle
    print_data = df.iloc[:, 15] #Concat printing string
    sens_data = df.iloc[:, 16] #Concat

    return [x_data, y_data, ele_data, mat_data, print_data, sens_data]

force = get_plotting_data(df_force)

# Get unique types for the different traces to make interactive legend
printing_processes = force[4].unique()
electrical_principles = force[2].unique()

sensing_basis = force[5].unique()
materials = force[3].unique()

# Create separate traces for each printing process and electrical principle
traces_left = []
for process in printing_processes:
    for principle in electrical_principles:
        filtered_data = force[0][(force[4] == process) & (force[2] == principle)], force[1][(force[4] == process) & (force[2] == principle)]
        if len(filtered_data[0]) > 0:  # Check if there's data for this combination
            trace = go.Scatter(
                x=filtered_data[0],
                y=filtered_data[1],
                mode='markers',
                marker=dict(
                    size=10,
                    symbol=marker_shape_map_printing_process.get(process, 'circle'),
                    color=get_color_from_map(principle, color_map),
                    line=dict(width=2, color='black')
                ),
                name=f'{process}',  # Only show process in legend
                legendgroup=f'{sensor_map_names[principle]}',  # Group by principle
                legendgrouptitle=dict(text=f'{sensor_map_names[principle]}'),
                visible=True  # Start with all traces hidden
            )
            traces_left.append(trace)
            fig_left.add_trace(trace)

traces_right = []
for basis in sensing_basis:
    for mat in materials:
        filtered_data = force[0][(force[5] == basis) & (force[3] == mat)], force[1][(force[5] == basis) & (force[3] == mat)]
        if len(filtered_data[0]) > 0:  # Check if there's data for this combination
            trace = go.Scatter(
                x=filtered_data[0],
                y=filtered_data[1],
                mode='markers',
                marker=dict(
                    size=10,
                    symbol=marker_shape_map_sensing_basis.get(basis, 'circle'),
                    color=get_color_from_map(mat, color_map),
                    line=dict(width=2, color='black')
                ),
                name=f'{basis}',  # Only show process in legend
                legendgroup=f'{material_map_names[mat]}',  # Group by material
                legendgrouptitle=dict(text=f'{material_map_names[mat]}'),
                visible=True  # Start with all traces hidden
            )
            traces_right.append(trace)
            fig_right.add_trace(trace)

# Update layout for legend columns
fig_left.update_layout(
    legend=dict(
        orientation="h",
        tracegroupgap=150,  # Adjust spacing as needed
        entrywidth=0.25,       # Adjust width as needed
        entrywidthmode="fraction" ,
        y=1.4, yanchor="top",
    ),
    legend_groupclick="toggleitem",
)

fig_right.update_layout(
    legend=dict(
        orientation="h",
        tracegroupgap=150,  # Adjust spacing as needed
        entrywidth=0.25,       # Adjust width as needed
        entrywidthmode="fraction" ,
        y=1.4, yanchor="top",
    ),
    legend_groupclick="toggleitem",
)


# Log scale axes
fig_left.update_xaxes(title_text="Force [N]",type='log')
fig_left.update_yaxes(title_text="Sensitivity [N<sup>-1</sup>]",type='log')
fig_right.update_xaxes(title_text="Force [N]",type='log')
fig_right.update_yaxes(title_text="Sensitivity [N<sup>-1</sup>]",type='log')

fig_left.update_layout(
    width = 700,
    height = 700,
)
fig_right.update_layout(
    width = 700,
    height = 700,
)
fig_left.update_layout(margin_autoexpand=False,margin_t=200)
fig_right.update_layout(margin_autoexpand=False,margin_t=200)

f_left = go.FigureWidget(fig_left)
f_right = go.FigureWidget(fig_right)

fig_force=  HBox([f_left, f_right])
fig_force